In [1]:
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
from collections import Counter

from collections import OrderedDict
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from html.parser import HTMLParser
from bs4 import BeautifulSoup

In [7]:
# nltk.download('stopwords')

In [2]:
porter = PorterStemmer()
wnl = WordNetLemmatizer() 
stop = stopwords.words('english')
stop.append("new")
stop.append("like")
stop.append("u")
stop.append("it'")
stop.append("'s")
stop.append("n't")
stop.append('mr.')
stop = set(stop)

In [9]:
# taken from http://ahmedbesbes.com/how-to-mine-newsfeed-data-and-extract-interactive-insights-in-python.html

def tokenizer(text):

    tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text)]

    tokens = []
    for token_by_sent in tokens_:
        tokens += token_by_sent

    tokens = list(filter(lambda t: t.lower() not in stop, tokens))
    tokens = list(filter(lambda t: t not in punctuation, tokens))
    tokens = list(filter(lambda t: t not in [u"'s", u"n't", u"...", u"''", u'``', u'\u2014', u'\u2026', u'\u2013'], tokens))
     
    filtered_tokens = []
    for token in tokens:
        token = wnl.lemmatize(token)
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)

    filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))

    return filtered_tokens


class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def get_keywords(tokens, num):
    return Counter(tokens).most_common(num)


def build_article_df(urls):
    articles = []
    for index, row in urls.iterrows():
        try:
            data=row['text'].strip().replace("'", "")
            data = strip_tags(data)
            soup = BeautifulSoup(data)
            data = soup.get_text()
            data = data.encode('ascii', 'ignore').decode('ascii')
            document = tokenizer(data)
            top_5 = get_keywords(document, 3)
          
            unzipped = zip(*top_5)
            kw= list(unzipped[0])
            kw=",".join(str(x) for x in kw)
            articles.append((kw, row['title'], row['pubdate']))
        except Exception as e:
            print( e)
            #print data
            #break
            pass
        #break
    article_df = pd.DataFrame(articles, columns=['keywords', 'title', 'pubdate'])
    return article_df



In [0]:
# df = pd.read_csv('../examples/tocsv.csv')

In [10]:
# df.head()

In [3]:
filename = './data/unhashed/iranian_tweets_csv_unhashed.csv'

In [4]:
df = pd.read_csv(filename, dtype = {
        "tweetid": str,
        "userid": str,
        "user_display_name": str,
        "user_screen_name": str,
        "user_reported_location": str,
        "user_profile_description": str,
        "user_profile_url": str,
        "follower_count": str,
        "following_count": str,
        "account_creation_date": str,
        "account_language": str,
        "tweet_text": str,
        "tweet_time": str,
        "tweet_client_name": str,
        "in_reply_to_tweetid": str,
        "in_reply_to_userid": str,
        "quoted_tweet_tweetid": str,
        "is_retweet": bool,
        "retweet_userid": str,
        "retweet_tweetid": str,
        "latitude": str,
        "longitude": str,
        "quote_count": np.float64,
        "reply_count": np.float64,
        "like_count": np.float64,
        "retweet_count": np.float64,
        "hashtags": str,
        "urls": str,
        "user_mentions": str,
        "poll_choices": str,
    })
# data = []
# for index, row in df.iterrows():
#     data.append((row['Title'], row['Permalink'], row['Date'], row['Content']))
# data_df = pd.DataFrame(data, columns=['title' ,'url', 'pubdate', 'text' ])

In [5]:
df = df[df['account_language'] == 'en']

In [11]:
# df.hashtags

In [ ]:
all_hashtags = []
for h in df.hashtags:
    

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,1), stop_words = 'english') # You can define your own parameters
X = cv.fit_transform(df.hashtags)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [14]:
# data_df.tail()

In [0]:
article_df = build_article_df(data_df)

/vagrant/Python/pythondata/env/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [0]:
article_df.head()

,keywords,title,pubdate
0,"data,big,culture",Building a Data Culture,20141118
1,"data,data-driven,company","Note to Self - Don't say ""Data Driven"" Anymore",20141120
2,"canon,captured,titmouse",Foto Friday - Titmouse on the Feeder,20141121
3,"mobility,organization,device",The Cloud - Gateway to Enterprise Mobility,20141121
4,"center,data,agile",The Agile Data Center,20141124


In [0]:
keywords_array=[]
for index, row in article_df.iterrows():
    keywords=row['keywords'].split(',')
    for kw in keywords:
        keywords_array.append((kw.strip(' '), row['keywords']))

kw_df = pd.DataFrame(keywords_array).rename(columns={0:'keyword', 1:'keywords'})

In [0]:
kw_df.head()

,keyword,keywords
0,data,"data,big,culture"
1,big,"data,big,culture"
2,culture,"data,big,culture"
3,data,"data,data-driven,company"
4,data-driven,"data,data-driven,company"


In [0]:
document = kw_df.keywords.tolist()
names = kw_df.keyword.tolist()



In [0]:
document_array = []
for item in document:
    items = item.split(',')
    document_array.append((items))

occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)

# Find the co-occurrences:
for l in document_array:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1


In [0]:
co_occur = pd.DataFrame.from_dict(occurrences )

In [0]:
co_occur.to_csv('out/ericbrown_co-occurancy_matrix.csv')

In [0]:
co_occur.head()

,data,big,culture,data-driven,company,canon,captured,titmouse,mobility,organization,...,multi-bracket,slow,science,knowledge,management,love,song,quality,dogmatic,pragmatic
300mm,0,0,0,0,0,3,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7d,0,0,0,0,0,18,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8-16mm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
agile,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
agility,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
